In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD   
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents(m).json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words2.pkl','wb'))
pickle.dump(classes,open('classes2.pkl','wb'))

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model2.h5', hist)

print("model created")


259 documents
37 classes ['address', 'canteen', 'certification', 'contact', 'coures-self-finance(pg)', 'courses-aided', 'courses-aided(number)', 'courses-pg(number)', 'courses-ug(number)', 'departments-pg', 'departments-ug', 'evening', 'facility', 'fees', 'fees-pg', 'fees-ug', 'goodbye', 'greeting', 'hostel', 'library', 'm.phil', 'morning', 'name', 'night', 'parking', 'payment-method', 'pg timings', 'phd', 'placements', 'placements-pg', 'placements-ug', 'principal', 'teachers', 'thanks', 'timings', 'ug timings', 'viceprincipal']
200 unique lemmatized words ["'s", ',', '.', ':', 'a', 'able', 'about', 'accept', 'accepted', 'accepts', 'address', 'aided', 'am', 'an', 'and', 'any', 'anyone', 'are', 'assisted', 'at', 'au', 'available', 'be', 'best', 'bonjour', 'by', 'bye', 'can', 'canteen', 'case', 'cash', 'cash-only', 'certificate', 'close', 'closing', 'college', 'come', 'company', 'contact', 'could', 'course', 'cure', 'department', 'detail', 'determined', 'do', 'doe', 'due', 'end', 'enroll

[nltk_data] Downloading package punkt to C:\Users\SPIRO-
[nltk_data]     PYTHON\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\SPIRO-
[nltk_data]     PYTHON\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\SPIRO-PYTHON\AppData\Local\Temp\ipykernel_5220\344142740.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


52/52 [==============================] - 0s 922us/step - loss: 3.5969 - accuracy: 0.0193
Epoch 2/200
52/52 [==============================] - 0s 902us/step - loss: 3.4289 - accuracy: 0.0965
Epoch 3/200
52/52 [==============================] - 0s 863us/step - loss: 3.1319 - accuracy: 0.1776
Epoch 4/200
52/52 [==============================] - 0s 902us/step - loss: 2.7762 - accuracy: 0.2625
Epoch 5/200
52/52 [==============================] - 0s 863us/step - loss: 2.4400 - accuracy: 0.3243
Epoch 6/200
52/52 [==============================] - 0s 863us/step - loss: 2.1301 - accuracy: 0.3938
Epoch 7/200
52/52 [==============================] - 0s 980us/step - loss: 1.8315 - accuracy: 0.5097
Epoch 8/200
52/52 [==============================] - 0s 1ms/step - loss: 1.6975 - accuracy: 0.4865
Epoch 9/200
52/52 [==============================] - 0s 1ms/step - loss: 1.4535 - accuracy: 0.5483
Epoch 10/200
52/52 [==============================] - 0s 902us/step - loss: 1.3334 - accuracy: 0.5753
Epoch

52/52 [==============================] - 0s 843us/step - loss: 0.2110 - accuracy: 0.9266
Epoch 82/200
52/52 [==============================] - 0s 863us/step - loss: 0.1797 - accuracy: 0.9305
Epoch 83/200
52/52 [==============================] - 0s 873us/step - loss: 0.2569 - accuracy: 0.9189
Epoch 84/200
52/52 [==============================] - 0s 863us/step - loss: 0.2780 - accuracy: 0.9112
Epoch 85/200
52/52 [==============================] - 0s 921us/step - loss: 0.1495 - accuracy: 0.9575
Epoch 86/200
52/52 [==============================] - 0s 2ms/step - loss: 0.1225 - accuracy: 0.9768
Epoch 87/200
52/52 [==============================] - 0s 2ms/step - loss: 0.1979 - accuracy: 0.9382
Epoch 88/200
52/52 [==============================] - 0s 1ms/step - loss: 0.1520 - accuracy: 0.9498
Epoch 89/200
52/52 [==============================] - 0s 922us/step - loss: 0.2062 - accuracy: 0.9228
Epoch 90/200
52/52 [==============================] - 0s 941us/step - loss: 0.2607 - accuracy: 0.9112

52/52 [==============================] - 0s 941us/step - loss: 0.1763 - accuracy: 0.9537
Epoch 162/200
52/52 [==============================] - 0s 961us/step - loss: 0.2067 - accuracy: 0.9382
Epoch 163/200
52/52 [==============================] - 0s 961us/step - loss: 0.1232 - accuracy: 0.9730
Epoch 164/200
52/52 [==============================] - 0s 961us/step - loss: 0.0816 - accuracy: 0.9807
Epoch 165/200
52/52 [==============================] - 0s 961us/step - loss: 0.1028 - accuracy: 0.9691
Epoch 166/200
52/52 [==============================] - 0s 863us/step - loss: 0.0793 - accuracy: 0.9846
Epoch 167/200
52/52 [==============================] - 0s 921us/step - loss: 0.1799 - accuracy: 0.9459
Epoch 168/200
52/52 [==============================] - 0s 941us/step - loss: 0.0820 - accuracy: 0.9691
Epoch 169/200
52/52 [==============================] - 0s 961us/step - loss: 0.1004 - accuracy: 0.9691
Epoch 170/200
52/52 [==============================] - 0s 882us/step - loss: 0.1093 - a